# Production mix - Model 2

## Situation
You own a boutique pottery business, making and selling two types of large ornamental products called Lunar Orb and Solar Disc.Given constraints on staff hours, available materials, and product sales, your objective is to maximize the total profit margin from the shop.

## Implementation
Linear Program (LP), using a Pyomo concrete model. Extracted previously hard-coded data into a dictionary and other data structures. Uses a DataFrame for output.

## Source
Replicates an Excel model described in article "Production mix via graphical LP" at https://www.solvermax.com/blog/production-mix.

In [1]:
# Import dependencies

import pyomo.environ as pyo
import pandas as pd

In [2]:
# Declarations

Model = pyo.ConcreteModel(name = 'Boutique pottery shop - Model 2')

Hours = 250
kg = 500
SalesLimit = 0

Coefficients = {
    'Discs' : {'People': 12.50, 'Materials': 18.00, 'Sales': -2.00, 'Margin':  80.00},  
    'Orbs'  : {'People': 10.00, 'Materials': 30.00, 'Sales':  1.00, 'Margin': 200.00},
}

Products = Coefficients.keys()

VarInitial = 0
VarBounds = (0, 100)

In [3]:
# Define model

Model.Production = pyo.Var(Products, domain = pyo.NonNegativeReals, initialize = VarInitial, bounds = VarBounds)

Model.PeopleHours = pyo.Constraint(expr = sum(Coefficients[p]['People'] * Model.Production[p] for p in Products) <= Hours)
Model.MaterialUsage = pyo.Constraint(expr = sum(Coefficients[p]['Materials'] * Model.Production[p] for p in Products) <= kg)
Model.SalesRelationship = pyo.Constraint(expr = sum(Coefficients[p]['Sales'] * Model.Production[p] for p in Products) <= SalesLimit)

Model.TotalMargin = pyo.Objective(expr = sum(Coefficients[p]['Margin'] * Model.Production[p] for p in Products), sense = pyo.maximize)

In [4]:
# Solve model

Solver = pyo.SolverFactory('cbc')
Results = Solver.solve(Model)

In [5]:
# Write output

print(Model.name, '\n')
print('Status: ', Results.solver.status, '\n')
print(f'Total margin = ${Model.TotalMargin():,.2f}\n')
ProductResults = pd.DataFrame()
for p in Products:
    ProductResults.loc[p, 'Production'] = round(pyo.value(Model.Production[p]), 2)
display(ProductResults)

Boutique pottery shop - Model 2 

Status:  ok 

Total margin = $3,076.92



,Production
Discs,6.41
Orbs,12.82
